In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import imblearn
from imblearn.over_sampling import SMOTE
import math
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
ecg_treino = pd.read_pickle("ecg_treino.pickle")
rotulos_treino = pd.read_csv("rotulos_treino.csv")
n_rotulos_treino = pd.read_csv("n_rotulos_treino.csv")  

ecg_teste = pd.read_pickle("ecg_teste.pickle")
rotulos_teste = pd.read_csv("rotulos_teste.csv")
n_rotulos_teste = pd.read_csv("n_rotulos_teste.csv") 

In [4]:
treino = ecg_treino.to_numpy()
r_treino = rotulos_treino.to_numpy()
n_r_treino = n_rotulos_treino.to_numpy()

teste = ecg_teste.to_numpy()
r_teste = rotulos_teste.to_numpy()
n_r_teste = n_rotulos_teste.to_numpy()

In [5]:
treino = np.delete(treino, 0, 1)
r_treino = np.delete(r_treino, 0, 1)
n_r_treino = np.delete(n_r_treino, 0, 1)

teste = np.delete(teste, 0, 1)
r_teste = np.delete(r_teste, 0, 1)
n_r_teste = np.delete(n_r_teste, 0, 1)

In [6]:
def n_rotulos_media(n_rotulos):
    intervalo_sum = 0
    for i in range(0, len(n_rotulos)):
        n_r_paciente = n_rotulos[i]
        n_r_adj = n_r_paciente[~pd.isna(n_r_paciente)]
        for j in range(0, len(n_r_adj)-1):
            intervalo_sum = intervalo_sum + abs(n_r_adj[j] - n_r_adj[j+1])
    n_r_media = round(intervalo_sum/(len(n_rotulos)*(len(n_r_adj)-1)))
    return n_r_media

In [12]:
def fourier(ecg,n_rotulos,n_r_media):
    num_batimentos = len(n_rotulos[:, ~pd.isna(n_rotulos).any(axis=0)][0])
    print(num_batimentos)
    matriz = np.zeros((num_batimentos*len(n_rotulos),n_r_media-1))
    for i in range(len(n_rotulos)):
        for j in range(0,num_batimentos-1):
            fourier = np.fft.fft(ecg[i, int(n_rotulos[i,j]) : int(n_rotulos[i,j+1])], n = n_r_media-1)
            matriz[i*num_batimentos+j] = fourier
    df = pd.DataFrame(matriz)
    return df, num_batimentos

In [11]:
n_r_media = n_rotulos_media(n_r_treino)
df, num_batimentos = fourier(treino,n_r_treino,n_r_media)
print(df)

1619


C:\Users\jean_\AppData\Local\Temp\ipykernel_5980\2283306410.py:8: ComplexWarning: Casting complex values to real discards the imaginary part
  matriz[i*num_batimentos+j] = fourier


           0         1          2          3          4          5    \
0      -84.505  1.368430   4.925815  12.255999   1.285518   3.952521   
1      -74.585  1.772179   6.271280  11.632441   1.491345   4.775133   
2      -86.055  3.114716   7.818601  13.948519   3.115563   6.436360   
3     -105.370  1.868015  10.261747  14.002971   4.475583   9.928012   
4      -93.695 -0.752889   8.633478  11.932169   4.405320   9.423333   
...        ...       ...        ...        ...        ...        ...   
35613  -30.025 -0.758627 -10.506144  11.224900  14.427000  10.032473   
35614  -45.905  4.006934  -7.161582  14.613097  18.393838   7.019992   
35615  -60.715  4.167397  -7.733947  14.711338  17.340752   8.631085   
35616  -34.930  1.274451  -8.983400  12.521675  18.386066   7.018106   
35617    0.000  0.000000   0.000000   0.000000   0.000000   0.000000   

             6         7          8          9    ...        277        278  \
0       4.661344  2.556563   3.984359   3.920407  ...   